# 03_Directed_Search_Performing_Experiments.ipynb

In this file we perform a large number of scenarios on the policies obtained from the previous step.

**Imports and Setup**

In [15]:
n_scenarios = pow(2, 1) #= 1024
pf_id = 2
name = "Rhys"

In [16]:
from ema_workbench import (ScalarOutcome, Scenario, MultiprocessingEvaluator, SequentialEvaluator)
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from ema_workbench.analysis import parcoords
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.util import ema_logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ema_logging.log_to_stderr(ema_logging.INFO)

model, planning_steps = get_model_for_problem_formulation(pf_id)


C:\Users\Alienware\OneDrive - Maastricht University\Documents\GitHub\EMA-Thesis\final assignment\funs_generate_network.py:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  return dict((str(k), v.dropna().to_dict())


In [17]:
directed_search_df = pd.read_csv('output/directed_search/Directed_Search_combined_1_results.csv')

**Re-evaluate candidate solutions under uncertainty**

We now have a large number of candidate solutions (policies), we can re-evaluate them over the various deeply uncertain factors to assess their robustness against uncertainties.

For this robustness evaluation, we perform 10000 experiments.

To reduce the run time, we will only be evaluating the best policies. We decided that those are the policies with zero deaths



In [18]:
directed_narrow = directed_search_df['Expected Number of Deaths'] <= 0.0 
directed_narrow = directed_search_df[directed_narrow]
directed_narrow

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths,Case
54,0,0,0,0,0,0,0,0,0,0,...,4,7,0,0,0.0,2.560100e+08,0.000000e+00,0.0,0.0,0
76,0,0,0,0,0,0,0,1,0,0,...,1,4,0,0,0.0,1.662552e+08,3.070000e+07,0.0,0.0,0
88,0,1,0,0,0,0,0,0,1,0,...,0,4,0,0,0.0,1.239637e+08,1.153000e+08,0.0,0.0,0
134,0,1,0,0,0,0,1,0,1,0,...,0,3,0,0,0.0,1.169251e+08,2.672000e+08,0.0,0.0,0
256,0,0,0,0,0,0,0,0,0,0,...,3,2,0,0,0.0,1.151080e+08,0.000000e+00,0.0,0.0,1
268,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0.0,2.503721e+07,1.519000e+08,0.0,0.0,1
278,0,0,0,0,0,0,1,1,0,0,...,0,2,0,0,0.0,3.389903e+07,6.140000e+07,0.0,0.0,1
285,0,0,0,0,0,0,0,1,0,0,...,2,3,0,0,0.0,8.515974e+07,3.070000e+07,0.0,0.0,1
291,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0.0,0.000000e+00,2.731000e+08,0.0,0.0,1
410,1,1,1,1,1,1,0,1,1,1,...,1,8,5,7,0.0,6.576607e+08,1.588300e+09,0.0,0.0,2


In [19]:
policies = directed_narrow.drop([o.name for o in model.outcomes], axis=1)
policies

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,Case
54,0,0,0,0,0,0,0,0,0,0,...,5,0,0,4,1,4,7,0,0,0
76,0,0,0,0,0,0,0,1,0,0,...,3,0,0,5,0,1,4,0,0,0
88,0,1,0,0,0,0,0,0,1,0,...,3,0,0,5,1,0,4,0,0,0
134,0,1,0,0,0,0,1,0,1,0,...,2,0,0,3,2,0,3,0,0,0
256,0,0,0,0,0,0,0,0,0,0,...,7,0,0,1,0,3,2,0,0,1
268,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,1,0,0,1
278,0,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,2,0,0,1
285,0,0,0,0,0,0,0,1,0,0,...,0,0,0,3,0,2,3,0,0,1
291,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
410,1,1,1,1,1,1,0,1,1,1,...,8,1,3,9,7,1,8,5,7,2


In [20]:
from ema_workbench import Policy

policies_to_evaluate = []

for i, policy in policies.iterrows():
    policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [21]:
with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies_to_evaluate)


[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 2 scenarios * 12 policies * 1 model(s) = 24 experiments
100%|██████████████████████████████████████████| 24/24 [00:48<00:00,  2.04s/it]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [30]:
outcomes = results[1]
experiments = results[0]

In [33]:
outcomes_df = pd.DataFrame.from_dict(outcomes)


In [34]:
outcomes_df.to_csv(f'./output/directed_search/Directed_Search_Scenario_Analysis_Outcomes_{name}.csv', index=True)
experiments.to_csv(f'./output/directed_search/Directed_Search_Scenario_Analysis_Experiments_{name}.csv', index=True)

In [35]:
directed_narrow.to_csv(f'./output/directed_search/Directed_Search_Scenario_Analysis_Policy_{name}.csv', index=True)